In [1]:
!pip -q install selenium==3.13.0
!pip -q install google-cloud-storage
!pip -q install gspread
!pip -q install gspread_dataframe
!pip -q install google-auth
!pip -q install google-api-python-client
!pip -q install python-dotenv
!pip -q install pytz
# !pip -q install google-cloud-secretmanager 本番環境のみ

In [5]:
#selenium関連
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

#データフレーム関連
import pandas as pd

#python関連
import re
import time
import pytz
from datetime import datetime

#Google Cloud Functions関連
import os
import zipfile
import shutil
from google.cloud import storage

#ログ出力関連
import json
import sys
from logging import getLogger, StreamHandler, DEBUG, INFO, Formatter
from logging import config

#Google API認証関連
import gspread
from google.oauth2 import service_account
from googleapiclient import discovery
from gspread_dataframe import set_with_dataframe

#環境変数関連
from env_local import (
    FILE_NAME, 
    RAKUTEN_URL,
    AMAZON_URL,
    YAHOO_URL,
    QOO10_URL,
    COSME_URL,
    LESAGE_URL,
    DRIVER_WAIT_TIME,
    SCROLL_PAUSE_TIME,
    MAX_SCROLL_ATTEMPTS,
    TIME_ZONE,
    TEMP_DIR,
    PROJECT_ID,
    SECRET_ID,
    VERSION_ID,
    BUCKET_NAME,
    ZIP_BLOB_NAME,
    ZIP_FILE,
    GOOGLE_DRIVE_FOLDER,
    GOOGLE_SPREADSHEET_PROPERTY,
    SCOPES
)

#カスタムロガーの設定
with open('logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

def download_blob(bucket_name, source_blob_name, destination_file_name):
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)
        logger.info(f'[source_blob_name]のダウンロードが完了しました')
    except Exception as e:
        logger.error(f'[source_blob_name]のダウンロード中にエラーが発生しました：[e]')
        raise
    
def setup_chrome_driver(binary_file_name):
    #optionの設定
    chrome_options = ChromeOptions()
    chrome_options.binary_location = binary_file_name
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--remote-debugging-port=9222")    
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1280x1696')
    chrome_options.add_argument('--hide-scrollbars')
    chrome_options.add_argument('--enable-logging')
    chrome_options.add_argument('--log-level=0')
    chrome_options.add_argument('--v=99')
    chrome_options.add_argument('--single-process')
    chrome_options.add_argument('--ignore-certificate-errors')
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
    chrome_options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})
    return chrome_options

def get_driver():
    try:
        with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
            zip_ref.extractall(TEMP_DIR)
            logger.info('chrome-headless.zipの解凍に成功しました')
    except Exception as e:
        logger.error(f'chrome-headless.zipの解凍に失敗しました:[e]')
        raise
    chrome_driver_path = os.path.join(TEMP_DIR,'chrome-headless', 'chromedriver')
    chrome_binary_path = os.path.join(TEMP_DIR,'chrome-headless', 'headless-chromium')
    logger.debug(f'ChromeDriverのパス: {chrome_driver_path}')
    logger.debug(f'ChromeDriverのパス: {chrome_binary_path}')
    # ChromeDriverのオプション設定
    options = setup_chrome_driver(chrome_binary_path)
    try:
        os.chmod(chrome_driver_path, 0o777)
        os.chmod(chrome_binary_path, 0o777)
        logger.info('chrome-headless配下フォルダに実行権限を付与しました')
    except Exception as e:
        logger.error('chrome-headless配下フォルダに実行権限が付与出来ませんでした')   
    try:
        driver = webdriver.Chrome(chrome_driver_path, options=options)
        logger.info('ChromeDriverを起動しました')
    except Exception as e:
        logger.error(f'ChromeDriverの起動に失敗しました: [e]')
        raise
    #サーバー負荷対策
    driver.implicitly_wait(DRIVER_WAIT_TIME)
    return driver


#-----楽天スクレイピング-----
def get_scraping_rakuten(driver, root, result_dict, web_site):
    #正規表現抽出
    def _get_rank_num(text):
        pattern = r'\d+(?=位)'
        rank_num = re.search(pattern, text).group()
        return rank_num
    
    def _get_price(text):
        pattern = r'(\d+,?\d*)'
        price = re.search(pattern, text).group()
        return price
    
    def _get_pagelink(page_links):
        a_elements = driver.find_elements(By.CSS_SELECTOR, ".pager a")
        for i, a_element in enumerate(a_elements):
            if i > 1:
                break
            link = a_element.get_attribute("href")
            page_links.append(link)
        page_link = page_links[0]
        logger.debug(f'{web_site}：ページリンクを取得しました')
        return page_link
    
    def _get_product_info_up_to_3():
        div_elements = driver.find_elements(By.CSS_SELECTOR,".rnkRanking_top3box")
        for div_element in div_elements:
            #順位取得
            img_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_rankIcon img")
            alt_value = img_element.get_attribute('alt')
            rank_num = _get_rank_num(alt_value)
            #商品名/URL取得
            a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_itemName a")
            prod_name = a_element.text
            prod_url = a_element.get_attribute('href')
            #会社名取得
            a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_shop a")
            company_name = a_element.text
            #価格取得
            div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_price")
            price_str = div_element_child.text
            price = _get_price(price_str)
            #結果格納
            result_dict[web_site].append(rank_num)
            result_dict['商品名'].append(prod_name)
            result_dict['会社名'].append(company_name)
            result_dict['価格'].append(price)
            result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：1～3位までの順位/商品名/会社名/価格/URLを取得しました')
        
    def _get_product_info_after_4(num_page):
        if num_page == 1:
            div_elements = driver.find_elements(By.CSS_SELECTOR,".rnkRanking_after4box")
            for div_element in div_elements:
                #順位取得
                div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_dispRank")
                div_element_child_str = div_element_child.text
                rank_num = _get_rank_num(div_element_child_str)
                #商品名/URL取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_itemName a")
                prod_name = a_element.text
                prod_url = a_element.get_attribute('href')
                #会社名取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_shop a")
                company_name = a_element.text
                #価格取得
                div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_price")
                price_str = div_element_child.text
                price = _get_price(price_str)
                #結果格納
                result_dict[web_site].append(rank_num)
                result_dict['商品名'].append(prod_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(prod_url)
        elif num_page == 2:
            div_elements = driver.find_elements(By.CSS_SELECTOR,".rnkRanking_after4box")
            for i, div_element in enumerate(div_elements, start=81):
                if i > 100:
                    break
                #順位取得
                if i == 100:
                    div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_dispRank_overHundred")
                else:
                    div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_dispRank")
                div_element_child_str = div_element_child.text
                rank_num = _get_rank_num(div_element_child_str)
                #商品名/URL取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_itemName a")
                prod_name = a_element.text
                prod_url = a_element.get_attribute('href')
                #会社名取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_shop a")
                company_name = a_element.text
                #価格取得
                div_element_child = div_element.find_element(By.CSS_SELECTOR,".rnkRanking_price")
                price_str = div_element_child.text
                price = _get_price(price_str)
                #結果格納
                result_dict[web_site].append(rank_num)
                result_dict['商品名'].append(prod_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：4位以降の順位/商品名/会社名/価格/URLを取得しました')
        
    #1ページ目へ遷移
    page_links = []
    num_page = 1
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目
    logger.debug('------1ページ目------')
    page_link = _get_pagelink(page_links)
    _get_product_info_up_to_3()
    _get_product_info_after_4(num_page)
    #2ページ目へ遷移
    num_page += 1
    try:
        driver.get(page_link)
        logger.debug(f'{web_site}：{num_page}ページ目へ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：{num_page}ページ目へ移動出来ませんでした: [e]')
        raise
    logger.debug(f'------{num_page}ページ目------')
    _get_product_info_after_4(num_page)
   
    #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    return result_dict


#-----Amazonスクレイピング-----
def get_scraping_amazon(driver, root, result_dict, web_site):
    #正規表現抽出
    def _get_company_name(text):
        # 特定の文字列を除く (ex:フェイスマスク)
        text = re.sub(r'フェイスマスク', '', text)
        #【】もしくは()とその中の文字列を除く
        text = re.sub(r'【[^】]*】|\([^)]*\)', '', text)
        # 例外文字列(半角が会社名に含まれている)のみ最初に抽出 (ex:ラ ロッシュ ポゼ)
        pattern =  r"ラ\s*ロッシュ\s*ポゼ"
        match = re.search(pattern, text)
        if match:
            return match.group(0)
        #文字列の先頭から単語文字または空白文字が1回以上続く部分を抽出
        # pattern =  r'^.*?(?=[【(])|^[\w\s]+'
        pattern =  r'^[\w\s]+'
        match = re.search(pattern, text)
        if match:
            result = match.group(0).split()[0]
            return result
    
    def _get_price(text):
        pattern = r'(\d+,?\d*)'
        price = re.search(pattern, text).group()
        return price
        
    def _get_pagelink():
        a_element = driver.find_element(By.CSS_SELECTOR, "li.a-normal a")
        page_link = a_element.get_attribute("href")
        logger.debug(f'{web_site}：ページリンクを取得しました')
        return page_link
    
    #スクロールしてページを更新
    def _scroll_page(driver):
        last_height = driver.execute_script("return document.body.scrollHeight")
        attempts = 0
        while attempts < 50:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            attempts += 1
    
    def _get_product_info():
        div_elements = driver.find_elements(By.ID,"gridItemRoot")
        for div_element in div_elements:
            #順位取得
            span_element = div_element.find_element(By.CSS_SELECTOR,".zg-bdg-text")
            span_element_str = span_element.text
            rank_num = span_element_str[1:]
            #商品名/会社名/URL取得
            a_element = div_element.find_element(By.CSS_SELECTOR,".a-link-normal:nth-of-type(2)")
            prod_url = a_element.get_attribute('href')
            prod_name = a_element.text
            company_name = _get_company_name(prod_name)
            #価格取得
            try:
                span_element = div_element.find_element(By.CSS_SELECTOR,"._cDEzb_p13n-sc-price_3mJ9Z")
                price_str = span_element.text
                price = _get_price(price_str)
            except NoSuchElementException:
                try:
                    span_element = div_element.find_element(By.CSS_SELECTOR,".p13n-sc-price")
                    price_str = span_element.text
                    price = _get_price(price_str)
                except NoSuchElementException:
                    price = "ー"
            #結果格納
            result_dict[web_site].append(rank_num)
            result_dict['商品名'].append(prod_name)
            result_dict['会社名'].append(company_name)
            result_dict['価格'].append(price)
            result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：順位/商品名/会社名/価格/URLを取得しました')         
            
    #1ページ目へ遷移
    num_page = 1
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目
    logger.debug('------1ページ目------')
    _scroll_page(driver)
    page_link = _get_pagelink()
    _get_product_info()
    #2ページ目へ遷移
    num_page += 1
    try:
        driver.get(page_link)
        logger.debug(f'{web_site}：{num_page}ページ目へ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：{num_page}ページ目へ移動出来ませんでした: [e]')
        raise
    logger.debug(f'------{num_page}ページ目------')
    _scroll_page(driver)
    _get_product_info()
   
    #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    return result_dict


#-----yahooショッピングスクレイピング-----
def get_scraping_yahoo(driver, root, result_dict, web_site):
    def _get_pagelink(page_links):
        a_elements = driver.find_elements(By.CSS_SELECTOR, "ul.elPager a")
        for a_element in a_elements:
            link = a_element.get_attribute("href")
            page_links.append(link)
        logger.debug(f'{web_site}：ページリンクを取得しました')
        return page_links
    
    def _get_product_info():
        li_elements = driver.find_elements(By.CSS_SELECTOR,"li.elItem.isReflect")
        for i, li_element in enumerate(li_elements, start=1):
            if i > 20:
                break
            #順位取得
            span_element = li_element.find_element(By.CSS_SELECTOR,".num")
            rank_num = span_element.text
            #商品名/URL取得
            a_element = li_element.find_element(By.CSS_SELECTOR,".elTitle a")
            prod_url = a_element.get_attribute('href')
            prod_name = a_element.text
            #会社名取得
            a_element = li_element.find_element(By.CSS_SELECTOR,".elStore.dcStoreIcon a")
            company_name = a_element.text
            #価格取得
            try:
                price = li_element.find_element(By.CSS_SELECTOR,".elPriceBody").text
            except NoSuchElementException:
                price = "ー"
            
            #結果格納
            result_dict[web_site].append(rank_num)
            result_dict['商品名'].append(prod_name)
            result_dict['会社名'].append(company_name)
            result_dict['価格'].append(price)
            result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：順位/商品名/会社名/価格/URLを取得しました')       
    
    #1ページ目へ遷移
    page_links = []
    num_page = 1
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目
    logger.debug('------1ページ目------')
    page_links = _get_pagelink(page_links)
    _get_product_info()
    #2ページ目以降へ遷移
    for page_link in page_links:
        num_page += 1
        try:
            driver.get(page_link)
            logger.debug(f'{web_site}：{num_page}ページ目へ移動しました')
        except Exception as e:
            logger.error(f'{web_site}：{num_page}ページ目へ移動出来ませんでした: [e]')
            raise
        logger.debug(f'------{num_page}ページ目------')
        _get_product_info()
    
    #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    
    return result_dict
    

#-----Qoo10スクレイピング-----
def get_scraping_qoo10(driver, root, result_dict, web_site):
    
    def _get_price(text):
        pattern = r'(\d+,?\d*)'
        price = re.search(pattern, text).group()
        return price
    
    def _get_product_info():
        div_elements = driver.find_elements(By.CSS_SELECTOR,"div.item")
        for i, div_element in enumerate(div_elements, start=1):
            if i > 100:
                break
            #順位取得
            span_element = div_element.find_element(By.CSS_SELECTOR,".rank")
            rank_num = span_element.text
            #商品名/URL取得
            a_element = div_element.find_element(By.CSS_SELECTOR,"a.tt")
            prod_url = a_element.get_attribute('href')
            prod_name = a_element.text
            #会社名取得
            try:
                a_element = div_element.find_element(By.CSS_SELECTOR,"a.txt_brand")
                company_name = a_element.text
            except NoSuchElementException:
                company_name = "ー"
            #価格取得
            strong_element = div_element.find_element(By.CSS_SELECTOR,".prc strong")
            price_str = strong_element.text
            price = _get_price(price_str)
            
            #結果格納
            result_dict[web_site].append(rank_num)
            result_dict['商品名'].append(prod_name)
            result_dict['会社名'].append(company_name)
            result_dict['価格'].append(price)
            result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：順位/商品名/会社名/価格/URLを取得しました') 
        
    #1ページ目へ遷移
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目
    logger.debug('------1ページ目------')
    _get_product_info()
        
     #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    
    return result_dict


#-----アットコスメスクレイピング-----
def get_scraping_cosme(driver, root, result_dict, web_site):
    #正規表現抽出
    def _get_rank_num(text):
        pattern = r'\d+(?=位)'
        rank_num = re.search(pattern, text).group()
        return rank_num
    
    def _get_price(text):
        pattern = r'(\d{1,3}(?:,\d{3})*)(?=円)'
        try:
            price = re.search(pattern, text).group(1)
        except Exception:
            price = "ー"
        return price
    
    def _get_pagelink(page_links):
        a_elements = driver.find_elements(By.CSS_SELECTOR, ".cmn-modules-paging li a")
        for i, a_element in enumerate(a_elements):
            if i > 8:
                break
            link = a_element.get_attribute("href")
            page_links.append(link)
        logger.debug(f'{web_site}：ページリンクを取得しました')
        return page_links
    
    def _get_product_info(num_page):
        if num_page == 1:
            div_elements = driver.find_elements(By.CSS_SELECTOR,"#list-item dl.clearfix")
            for i, div_element in enumerate(div_elements):
                #順位取得
                span_element = div_element.find_element(By.CSS_SELECTOR,".rank-num")
                if i < 3:
                    img_element = span_element.find_element(By.CSS_SELECTOR,"img")
                    alt_value = img_element.get_attribute('alt')
                    rank_num = _get_rank_num(alt_value)
                else:
                    span_element_child = span_element.find_element(By.CSS_SELECTOR,".num")
                    rank_num = span_element_child.text
                #商品名/URL取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".item a")
                prod_name = a_element.text
                prod_url = a_element.get_attribute('href')
                #会社名取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".brand a:nth-of-type(1)")
                company_name = a_element.text
                #価格取得
                p_element = div_element.find_element(By.CSS_SELECTOR,".price")
                price_str = p_element.text
                price = _get_price(price_str)
                #結果格納
                result_dict[web_site].append(rank_num)
                result_dict['商品名'].append(prod_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(prod_url)
        else:
            div_elements = driver.find_elements(By.CSS_SELECTOR,"#list-item dl.clearfix")
            for div_element in div_elements:
                #順位取得
                span_element = div_element.find_element(By.CSS_SELECTOR,".rank-num")
                span_element_child = span_element.find_element(By.CSS_SELECTOR,".num")
                rank_num = span_element_child.text
                #商品名/URL取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".item a")
                prod_name = a_element.text
                prod_url = a_element.get_attribute('href')
                #会社名取得
                a_element = div_element.find_element(By.CSS_SELECTOR,".brand a:nth-of-type(1)")
                company_name = a_element.text
                #価格取得
                p_element = div_element.find_element(By.CSS_SELECTOR,".price")
                price_str = p_element.text
                price = _get_price(price_str)
                #結果格納
                result_dict[web_site].append(rank_num)
                result_dict['商品名'].append(prod_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(prod_url)
        logger.debug(f'{web_site}：順位/商品名/会社名/価格/URLを取得しました') 
       
    #1ページ目へ遷移
    page_links = []
    num_page = 1
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目
    logger.debug('------1ページ目------')
    page_links = _get_pagelink(page_links)
    _get_product_info(num_page)
    #2ページ目以降へ遷移
    for page_link in page_links:
        num_page += 1
        try:
            driver.get(page_link)
            logger.debug(f'{web_site}：{num_page}ページ目へ移動しました')
        except Exception as e:
            logger.error(f'{web_site}：{num_page}ページ目へ移動出来ませんでした: [e]')
            raise
        logger.debug(f'------{num_page}ページ目------')
        _get_product_info(num_page)
    
    #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    
    return result_dict
        

#-----レサージュ-----
def get_scraping_lesage(driver, root, result_dict, web_site):    
    #正規表現抽出
    def _get_company_name(text):
        # 特定の文字列を除く (ex:お買い得 )
        text = re.sub(r'お買い得 ', '', text)
        #【】・()・［］とその中の文字列を除く
        text = re.sub(r'【[^】]*】|\([^)]*\)|［[^］]*］', '', text)
        # 例外文字列のみ最初に抽出
        patterns =  [
            r"Ogshi",
            r"資生堂",
            r"ロート製薬",
            r"ラッシュアディクト",
            r"プラスリストア",
            r"ウルトラV",
            r"Lov me Touch",
            r"アプローラ",
            r"リビジョン スキンケア",
            r"スキン52"
        ]
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                return match.group(0)
        #文字列の先頭からひらがなorカタカナが1回以上続く部分を抽出
        # pattern =  r'^.*?(?=[【(])|^[\w\s]+'
        pattern = r"(?!\[.*?\])[\u3040-\u30FF]+"
        match = re.search(pattern, text)
        if match:
            result = match.group(0).split()[0]
            return result
        
    def _get_pagelink(page_links):
        a_elements = driver.find_elements(By.CSS_SELECTOR, "ul.ec-pager a")
        for i, a_element in enumerate(a_elements):
            if i > 1:
                break
            link = a_element.get_attribute("href")
            page_links.append(link)
        page_link = page_links[-1]
        logger.debug(f'{web_site}：ページリンクを取得しました')
        return page_link
    
    def _get_product_info(num_page):
        if num_page == 1:
            a_elements = driver.find_elements(By.CSS_SELECTOR,"li.ec-shelfGrid__item a")
            for i, a_element in enumerate(a_elements, start=1):
                #※順位はiにて取得
                #URL取得
                href_value = a_element.get_attribute('href')
                #商品名/会社名取得
                p_element = a_element.find_element(By.CSS_SELECTOR,"p:nth-of-type(2)")
                product_name = p_element.text
                company_name = _get_company_name(product_name)
                #価格取得
                price = a_element.find_element(By.CSS_SELECTOR,".c-price").text
                #結果格納
                result_dict[web_site].append(i)
                result_dict['商品名'].append(product_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(href_value)    
        elif num_page == 2:
            a_elements = driver.find_elements(By.CSS_SELECTOR,"li.ec-shelfGrid__item a")
            for i, a_element in enumerate(a_elements, start=61):
                if i > 100:
                    break
                #※順位はiにて取得
                #URL取得
                href_value = a_element.get_attribute('href')
                #商品名/会社名取得
                p_element = a_element.find_element(By.CSS_SELECTOR,"p:nth-of-type(2)")
                product_name = p_element.text
                company_name = _get_company_name(product_name)
                #価格取得
                price = a_element.find_element(By.CSS_SELECTOR,".c-price")
                #結果格納
                result_dict[web_site].append(i)
                result_dict['商品名'].append(product_name)
                result_dict['会社名'].append(company_name)
                result_dict['価格'].append(price)
                result_dict['URL'].append(href_value)        
        logger.debug(f'{web_site}：順位/商品名/会社名/価格/URLを取得しました。')
    
    #1ページ目へ遷移
    page_links = []
    num_page = 1
    try:
        driver.get(root)
        logger.debug(f'{web_site}：トップページへ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：トップページへ移動出来ませんでした: [e]')
    #1ページ目(1位～60位)
    logger.debug('------1ページ目------')
    page_link = _get_pagelink(page_links)
    _get_product_info(num_page)
    #2ページ目以降へ遷移
    num_page += 1
    try:
        driver.get(page_link)
        logger.debug(f'{web_site}：{num_page}ページ目へ移動しました')
    except Exception as e:
        logger.error(f'{web_site}：{num_page}ページ目へ移動出来ませんでした: [e]')
        raise
    logger.debug(f'------{num_page}ページ目------')
    _get_product_info(num_page)
    
    #取得データ確認
    logger.debug(f'------取得結果------')
    logger.debug(f'{web_site}："{web_site}" {len(result_dict[f"{web_site}"])}個')
    logger.debug(f'{web_site}："商品名" {len(result_dict["商品名"])}個')
    logger.debug(f'{web_site}："会社名" {len(result_dict["会社名"])}個')
    logger.debug(f'{web_site}："価格" {len(result_dict["価格"])}個')
    logger.debug(f'{web_site}："URL" {len(result_dict["URL"])}個')
    
    return result_dict

def post_pd_to_sheet(df, execution_time):
    # サービスアカウントキーを使って認証
    SERVICE_ACCOUNT_FILE = '/workspace/extended-study-382401-8676076ee48a.json'
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    # gspreadのクライアントとGoogle Drive APIクライアントを設定
    gc = gspread.authorize(credentials)
    # Google Sheets API clientを作成
    service = discovery.build('sheets', 'v4', credentials=credentials)
    # Google Drive API clientを作成
    drive_service = discovery.build('drive', 'v3', credentials=credentials)
    # Google DriveフォルダIDを指定
    folder_id = GOOGLE_DRIVE_FOLDER
    page_token = None
    items = []
    
    # フォルダ内のスプレッドシートを検索
    try:
        condition_list = [
            'mimeType="application/vnd.google-apps.spreadsheet"',
            "trashed = false",
            f"'{folder_id}' in parents" 
        ]
        conditions = " and ".join(condition_list)
        while True:
            results = drive_service.files().list(
                q=conditions,
                fields="nextPageToken, files(id, name)"
            ).execute()
            items.extend(results.get("files", []))
            page_token = results.get("nextPageToken", None)
            if page_token is None:
                break
        logger.info('データ取得フォルダのファイル一覧を取得しました')
    except Exception as e:
        logger.error('データ取得フォルダのファイル一覧が取得出来ませんでした。')
        raise
    
    #フォルダ内の各スプレッドシートにデータフレームを転記
    is_updated = False
    template_sheet_name = 'template'
    new_sheet_name = f'{execution_time.hour}時取得'
    for item in items:
        #後続の日付が変わった場合のコピー元idを取得
        if f'{FILE_NAME}' in item['name']:
            spreadsheet_id = item['id']
        if f'【{execution_time.strftime("%Y-%m-%d")}】{FILE_NAME}' in item['name']:
            spreadsheet_id = item['id']
            spreadsheet = gc.open_by_key(spreadsheet_id)
            template_sheet = spreadsheet.worksheet(template_sheet_name)
            try:
                new_sheet = spreadsheet.worksheet(new_sheet_name)
                set_with_dataframe(new_sheet, df)
                logger.info(f"シート {new_sheet_name} はすでに存在しているため上書きしました。")
            except gspread.exceptions.WorksheetNotFound:
                new_sheet = spreadsheet.duplicate_sheet(template_sheet.id, new_sheet_name=new_sheet_name)
                set_with_dataframe(new_sheet, df)
                logger.info(f"シート {new_sheet_name} を新規作成しました。")
            is_updated = True
            break
    #日付が変わった場合の処理
    if not is_updated:
        new_file_name = f'【{execution_time.strftime("%Y-%m-%d")}】{FILE_NAME}'
        body = {
            'name': new_file_name,
            'parents':[folder_id]
        }
        try:
            new_spreadsheet = drive_service.files().copy(fileId=spreadsheet_id, body=body).execute()
            new_spreadsheet_id = new_spreadsheet.get('id')
            logger.info(f'新しいファイル {new_file_name} を作成しました。')
        except Exception as e:
            logger.error(f'新しい日付ファイルが作成できませんでした。: {e}')
            new_spreadsheet_id = None
            raise
        # 新しくコピーされた日付ファイルへ転記
        if new_spreadsheet_id:
            spreadsheet = gc.open_by_key(new_spreadsheet_id)
        # templateと0時ファイル以外を削除
        for other_sheet in spreadsheet.worksheets():
            if other_sheet.title != 'template' and other_sheet.title != new_sheet_name:
                spreadsheet.del_worksheet(other_sheet)
        new_sheet = spreadsheet.worksheet(new_sheet_name)
        set_with_dataframe(new_sheet, df)
        
    # 新シートの表示設定を非表示→表示 + 末尾へ移動
    sheets_count = len(spreadsheet.worksheets())
    GOOGLE_SPREADSHEET_PROPERTY['requests'][0]['updateSheetProperties']['properties']['sheetId'] = new_sheet.id
    GOOGLE_SPREADSHEET_PROPERTY['requests'][1]['updateSheetProperties']['properties']['sheetId'] = new_sheet.id
    GOOGLE_SPREADSHEET_PROPERTY['requests'][1]['updateSheetProperties']['properties']['index'] = sheets_count
    body = GOOGLE_SPREADSHEET_PROPERTY
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()
    logger.info('GoogleSpreadSheetへ転記しました')

def main():
    logger.info('------スクレイピングを開始します------')    
    #実行時間取得
    jst = pytz.timezone(TIME_ZONE)
    execution_time = datetime.now(jst)
    try:
        driver = get_driver()
        logger.info('ドライバ準備完了')
    except Exception as e:
        logger.error(f'ドライバ準備に失敗しました: [{e}]')
        raise
    
    web_sites = ["楽天", "Amazon", "ヤフーショッピング", "Qoo10", "アットコスメ", "レサージュ"]
    for web_site in web_sites:
        result_dict = {
            f'{web_site}': [], 
            '商品名': [], 
            '会社名': [],
            '価格':[],
            'URL': [],
        }
        if web_site == "楽天":
            try:
                logger.info('------楽天取得開始------')
                result_dict_rakuten = get_scraping_rakuten(driver, RAKUTEN_URL, result_dict, web_site)
                logger.info('------楽天取得完了------')
            except Exception as e:
                logger.error(f'楽天取得に失敗しました: [{e}]')
                driver.quit()
                raise
        elif web_site == "Amazon":
            try:
                logger.info('------AMAZON取得開始------')
                result_dict_amazon = get_scraping_amazon(driver, AMAZON_URL, result_dict, web_site)
                logger.info('------AMAZON取得完了------')
            except Exception as e:
                logger.error(f'AMAZON取得に失敗しました: [{e}]')
                driver.quit()
                raise
        elif web_site == "ヤフーショッピング":
            try:
                logger.info('------Yahoo取得開始------')
                result_dict_yahoo = get_scraping_yahoo(driver, YAHOO_URL, result_dict, web_site)
                logger.info('------Yahoo取得完了------')
            except Exception as e:
                logger.error(f'Yahoo取得に失敗しました: [{e}]')
                driver.quit()
                raise
        elif web_site == "Qoo10":
            try:
                logger.info('------QOO10取得開始------')
                result_dict_qoo10 = get_scraping_qoo10(driver, QOO10_URL, result_dict, web_site)
                logger.info('------QOO10取得完了------')
            except Exception as e:
                logger.error(f'QOO10取得に失敗しました: [{e}]')
                driver.quit()
                raise
        elif web_site == "アットコスメ":
            try:
                logger.info('------アットコスメ取得開始------')
                result_dict_cosme = get_scraping_cosme(driver, COSME_URL, result_dict, web_site)
                logger.info('------アットコスメ取得完了------')
            except Exception as e:
                logger.error(f'アットコスメ取得に失敗しました: [{e}]')
                driver.quit()
                raise
        elif web_site == "レサージュ":
            try:
                logger.info('------レサージュ取得開始------')
                result_dict_lesage = get_scraping_lesage(driver, LESAGE_URL, result_dict, web_site)
                logger.info('------レサージュ取得完了------')
            except Exception as e:
                logger.error(f'レサージュ取得に失敗しました: [{e}]')
                driver.quit()
                raise
    #ドライバ解放
    driver.quit()
    
    #転記用データフレーム作成
    result_df_rakuten = pd.DataFrame(result_dict_rakuten)
    result_df_amazon = pd.DataFrame(result_dict_amazon)
    result_df_yahoo = pd.DataFrame(result_dict_yahoo)
    result_df_qoo10 = pd.DataFrame(result_dict_qoo10)
    result_df_cosme = pd.DataFrame(result_dict_cosme)
    result_df_lesage = pd.DataFrame(result_dict_lesage)
    
    # データフレーム結合
    result_df = pd.concat([
        result_df_rakuten,
        result_df_amazon,
        result_df_yahoo,
        result_df_qoo10,
        result_df_cosme,
        result_df_lesage
    ],axis=1)
    
    try:
        post_pd_to_sheet(result_df, execution_time)
        logger.info('GoogleSpreadSheetへの転記が完了しました')
    except Exception as e:
        logger.error(f'GoogleSpreadSheetへの転記に失敗しました:[{e}]')
        raise
    logger.info('------スクレイピングを終了します------')
    
    return result_df
    
result = main()
result

2023-04-10 00:06:32,084 [INFO] main: ------スクレイピングを開始します------
2023-04-10 00:06:32,866 [INFO] main: chrome-headless.zipの解凍に成功しました
2023-04-10 00:06:32,868 [DEBUG] main: ChromeDriverのパス: /workspace/chrome-headless/chromedriver
2023-04-10 00:06:32,868 [DEBUG] main: ChromeDriverのパス: /workspace/chrome-headless/headless-chromium
2023-04-10 00:06:32,876 [INFO] main: chrome-headless配下フォルダに実行権限を付与しました
2023-04-10 00:06:34,025 [INFO] main: ChromeDriverを起動しました
2023-04-10 00:06:34,027 [INFO] main: ドライバ準備完了
2023-04-10 00:06:34,028 [INFO] main: ------楽天取得開始------
2023-04-10 00:06:35,472 [DEBUG] main: 楽天：トップページへ移動しました
2023-04-10 00:06:35,473 [DEBUG] main: ------1ページ目------
2023-04-10 00:06:35,592 [DEBUG] main: 楽天：ページリンクを取得しました
2023-04-10 00:06:36,075 [DEBUG] main: 楽天：1～3位までの順位/商品名/会社名/価格/URLを取得しました
2023-04-10 00:06:45,082 [DEBUG] main: 楽天：4位以降の順位/商品名/会社名/価格/URLを取得しました
2023-04-10 00:06:46,014 [DEBUG] main: 楽天：2ページ目へ移動しました
2023-04-10 00:06:46,015 [DEBUG] main: ------2ページ目------
2023-04-10 00:06:48,228 [

,楽天,商品名,会社名,価格,URL,Amazon,商品名,会社名,価格,URL,...,アットコスメ,商品名,会社名,価格,URL,レサージュ,商品名,会社名,価格,URL
0,1,【クーポンで679円】シートマスク プラセンタエキス等50%配合 30枚入り ホワイト オー...,クオリティファースト 楽天市場店,"2,508",https://item.rakuten.co.jp/q1st/10000002/?l2-i...,1,VTCOSMETICS(ブイティコスメテックス) 【正規品】シカデイリースージングマスク フ...,VTCOSMETICS,"2,420",https://www.amazon.co.jp/VTCOSMETICS-%E3%83%96...,...,1,リップモンスター,ケイト,880,https://www.cosme.net/product/product_id/10206...,1,ロート製薬 DRX 保湿乳液 ADパーフェクトバリア ボディミルク 130ml,ロート製薬,"1,840",https://www.lesage-store.jp/products/detail/417
1,2,【50％～51%OFFクーポン】680円(2点購入での1点あたり) 半額 パック シートマス...,プリュ公式ショップ 楽天市場店,"1,390",https://item.rakuten.co.jp/luire/pmm-yami/?l2-...,2,イニスフリー (innisfree) ノーセバム ミネラルパウダー N 5グラム (x 1),イニスフリー,825,https://www.amazon.co.jp/%E3%82%A4%E3%83%8B%E3...,...,2,リポソーム アドバンスト リペアセラム,コスメデコルテ,"8,250",https://www.cosme.net/product/product_id/10209...,2,ロート製薬 DRX ハイドロキノンクリーム HQダブルブライトE,ロート製薬,"2,000",https://www.lesage-store.jp/products/detail/411
2,3,【アテニア 公式】スキンクリア クレンズ オイル エコパック(全2種) 送料無料[Atten...,アテニア公式ショップ 楽天市場店,"3,300",https://item.rakuten.co.jp/attenir/166013/?l2-...,3,メラノCC ディープクリア酵素洗顔 130g 酵素×ビタミンC配合 洗顔フォーム 毛穴ケア,メラノCC,715,https://www.amazon.co.jp/%E3%80%90%E5%85%88%E8...,...,3,リポソーム アドバンスト リペアクリーム,コスメデコルテ,"11,000",https://www.cosme.net/product/product_id/10225...,3,ロート製薬 DRX 保湿乳液 ADパーフェクトバリア フェイスミルク 50ml,ロート製薬,"1,840",https://www.lesage-store.jp/products/detail/418
3,4,[エントリー最大100%ポイントバック+エントリーP14倍10日9:59マデ]Yunth(ユ...,Yunth Online Store 楽天市場店,"3,960",https://item.rakuten.co.jp/yunth/10000000/?l2-...,4,アテニア (Attenir) スキンクリア クレンズオイル アロマタイプ [ レギュラーボト...,アテニア,"1,870",https://www.amazon.co.jp/Attenir-%E3%82%B9%E3%...,...,4,ディオール アディクト リップ マキシマイザー,ディオール,"4,620",https://www.cosme.net/product/product_id/10233...,4,ZO SKIN HEALTH ゼオスキンヘルス バランサートナー,ゼオスキンヘルス,"6,400",https://www.lesage-store.jp/products/detail/78
4,5,【アテニア 公式】スキンクリア クレンズ オイル (レギュラーボトル)(全2種) 送料無料[...,アテニア公式ショップ 楽天市場店,"1,870",https://item.rakuten.co.jp/attenir/166011/?l2-...,5,ビオレUＶ アクアリッチ アクアプロテクトミスト 60ミリリットル (x 1),ビオレUＶ,891,https://www.amazon.co.jp/%E3%83%93%E3%82%AA%E3...,...,5,クリーミータッチライナー,キャンメイク,715,https://www.cosme.net/product/product_id/10147...,5,NEW プラスリストアシリーズ plus RESTORE UVローション（日焼け止め）SPF...,プラスリストア,"2,800",https://www.lesage-store.jp/products/detail/303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,★ポイント10倍＆割引クーポン★コスメデコルテ リポソーム アドバンスト リペアセラム 10...,コスメ ヴィーナス 楽天市場店,"12,870",https://item.rakuten.co.jp/cosme-venus/4971710...,96,ビオレ ザフェイス ディープモイスト つめかえ用 340ml(約2.1回分)【泡洗顔】【まさ...,ビオレ,"1,078",https://www.amazon.co.jp/%E3%83%93%E3%82%AA%E3...,...,96,ナチュラル チークN,セザンヌ,396,https://www.cosme.net/product/product_id/27441...,96,Revision Skincare リビジョン スキンケア D.E.J face cream...,リビジョン スキンケア,<selenium.webdriver.remote.webelement.WebEleme...,https://www.lesage-store.jp/products/detail/336
96,97,【レビューNo.1】【国内正規品】ザ プロダクト ヘアワックス 42g 1個 product...,ビューファ,"1,503",https://item.rakuten.co.jp/mobility2/r180503-01n/,97,【先行販売】ビオレ ザクレンズ オイルメイク落とし 本体 190ml クレンジングオイル ク...,ビオレ,"1,192",https://www.amazon.co.jp/%E3%80%90%E5%85%88%E8...,...,97,パドル ブラシ,AVEDA(アヴェダ),"3,740",https://www.cosme.net/product/product_id/34756...,97,［デオドラント シリーズ］D-bar（ディーバー）制汗スティック,ディーバー,<selenium.webdriver.remote.webelement.WebEleme...,https://www.lesage-store.jp/products/detail/33
97,98,【アテニア 公式】スキンクリア クレンズ オイル エコパック＋ポンプセット(全2種) 送料無...,アテニア公式ショップ 楽天市場店,"3,352",https://item.rakuten.co.jp/attenir/166103/,98,大洋 製薬 ワセリンHG クリーム 単品 100グラム (x 1),大洋,784,https://www.amazon.co.jp/%E5%A4%A7%E6%B4%8B%E8...,...,98,クリア サンケア スティック,SHISEIDO,"3,080",https://www.cosme.net/product/product_id/10206...,98,人気商品 お買い得 ［お買い得セット］HERRAS ミルキーピール エムディーソープ 乳酸菌...,ミルキーピール,<selenium.webdriver.remote.webelement.WebEleme...,https://www.lesage-store.jp/products/detail/882
98,99,【国産クレイ配合！ お得な3個セット 送料無料 W洗顔不要 とろけるクレンジング】ink. ...,ink.オンラインストア楽天市場店,"2,980",https://item.rakuten.co.jp/129-ink/ink90-3/,99,【医薬部外品】ビオレ マシュマロホイップ 薬用アクネケア つめかえ用 大容量 洗顔 さわやか...,ビオレ,"1,089",https://www.amazon.co.jp/%E3%80%90%E5%8C%BB%E8...,...,99,マイラ